In [ ]:
#| default_exp project

# project

> A simple API for handling time spent in a project

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from fastcore.utils import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [ ]:
#| export
class __Project():
    "Project (time) data handler"
    def __init__(self,
                 scopes: list, # google apis authentication scopes,
                 fn: str, # service account file name
                 sid: str): # spreadsheet id
        creds = service_account.Credentials.from_service_account_file(fn, scopes=scopes)
        service = build('sheets', 'v4', credentials=creds)
        self.sheet = service.spreadsheets()
        self.sid = sid
    def __str__(self): return f"{self.sid}"
    __repr__=__str__

In [ ]:
#| export
SERVICE_ACCOUNT_FILE = '../keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1DNoNf4glcuMxKoVzHVrFo-MktmsVji1wf4IHeraWH84'

class Project(__Project):
    def __init__(self,
                fn: str): # service account file name
        super(Project, self).__init__(SCOPES, SERVICE_ACCOUNT_FILE, SPREADSHEET_ID)

Project() prints spread sheet id, `sid`.

In [ ]:
pr = Project(SERVICE_ACCOUNT_FILE)

In [ ]:
#| export
@patch
def get(self:Project,
       sname:str=""): # a worksheet name can be specified. If not the 1st sheet is choosen.
    range = f"{sname}!" if sname else ""
    range += "A1:Z1000"
    result = self.sheet.values().get(spreadsheetId=self.sid,
                                range=range).execute()
    values = result.get('values', [])
    return values

For instance, here's some tests of equality

In [ ]:
pr.get("Test")

[['3/1/2022', '4000'], ['4/4/2022', '3000'], ['7/12/2022', '7000']]

In [ ]:
#| export
@patch
def put(self:Project,
        data:list,
        sname:str=""): # a worksheet name can be specified. If not the 1st sheet is choosen.
    range = f"{sname}!" if sname else ""
    range += "A1"
    request = self.sheet.values().update(spreadsheetId=self.sid,
                                         range=range,
                                         valueInputOption="USER_ENTERED",
                                         body={"values":data})
    response = request.execute()

In [ ]:
data = [["3/1/2022", "4000"],["4/4/2022", "3000"],["7/12/2022", "7000"]]
pr.put(data, "Test")
pr.get("Test")
test_eq(pr.get("Test"), data)

In [ ]:
pr.get()

[['name', 'backlog item #', 'hours spent', 'date'],
 ['Hiroshi Doyu', '25', '1', '2022/1/16'],
 ['Michihito Mizutani', '28', '1', '2022/1/16']]